In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# EDA with python and applying Logistic Regression

For this lecture we will be working with the Titanic Data Set from Kaggle. This is a very famous data set and very often is a student's first step in machine learning!

We'll be trying to predict a classification- survival or deceased. Let's begin our understanding of implementing Logistic Regression in Python for classification.

We'll use a "semi-cleaned" version of the titanic data set, if you use the data set hosted directly on Kaggle, you may need to do some additional cleaning not shown in this lecture notebook.


In [ ]:
filename='/kaggle/input/titanic-machine-learning-from-disaster/train.csv'
df=pd.read_csv(filename,encoding='ISO-8859-1')
df.head() 


# Exploratory data analysis

start by checking missing values


# Missing values
check for null values

In [ ]:
df.isnull()

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False)

In [ ]:
df.isnull().sum()

In [ ]:
sns.countplot("Survived",data=df)

In [ ]:
#style=white, dark, whitegrid, darkgrid, ticks
sns.set_style('whitegrid')
sns.countplot("Survived",data=df)

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=df,palette='rainbow')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=df,palette='rainbow')

In [ ]:
sns.distplot(df['Age'].dropna(),kde=True,color='green',bins=30)

In [ ]:
df['Age'].hist(bins=40,color='darkred',alpha=0.5)

In [ ]:
sns.countplot('SibSp',data=df)

In [ ]:
df['Fare'].hist(color='green',bins=40,figsize=(8,4))

In [ ]:
df['Parch'].hist(color='red',bins=40,figsize=(8,4))


# Data Cleaning

We want to fill in missing age data instead of just dropping the missing age data rows. One way to do this is by filling in the mean age of all the passengers (imputation). However we can be smarter about this and check the average age by passenger class.


In [ ]:
plt.figure(figsize=(12,7))
sns.boxplot(x='Pclass',y='Age',data=df,palette='winter')

In [ ]:
#We'll use these average age values to impute based on Pclass for Age.

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age

In [ ]:
df['Age'] = df[['Age','Pclass']].apply(impute_age,axis=1)

In [ ]:
sns.heatmap(df.isnull(),yticklabels='False')

In [ ]:
df.isnull().sum()

In [ ]:
df.drop('Cabin',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False)


# Converting Categorical Features

We'll need to convert categorical features to dummy variables using pandas! Otherwise our machine learning algorithm won't be able to directly take in those features as inputs.


In [ ]:
df.info()

In [ ]:
pd.get_dummies(df['Embarked'],drop_first=True).head()

In [ ]:
sex = pd.get_dummies(df['Sex'],drop_first=True)
embark = pd.get_dummies(df['Embarked'],drop_first=True)

In [ ]:
df.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
train = pd.concat([df,sex,embark],axis=1)

In [ ]:
train.head()


# Building a Logistic Regression model

Let's start by splitting our data into a training set and test set (there is another test.csv file that you can play around with in case you want to use all this data for training).


# Train Test Split

In [ ]:
X=train.drop('Survived',axis=1)
X.head()

In [ ]:
y=train['Survived']
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,  random_state=101)

# Training and Predicting

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

In [ ]:
predictions = logmodel.predict(X_test)

In [ ]:
predictions

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
accuracy=confusion_matrix(y_test,predictions)

In [ ]:
accuracy

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy=accuracy_score(y_test,predictions)
accuracy

In [ ]:
predictions

# Evaluation

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,predictions))